## <center>Analiza poziomu PM2.5 w afrykańskich miastach</center>
### Zespół:
<ol>
    <li style='font-size: 20px'>Hubert Kłosowski 242424</li>
    <li style='font-size: 20px'>Krzysztof Kolanek 242425</li>
    <li style='font-size: 20px'>Kamil Małecki 242464</li>
</ol>

### Potrzebne importy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Wczytanie danych

In [ ]:
data = pd.read_csv('data\\Train.csv')
test = pd.read_csv('data\\Test.csv')

data.info()

In [ ]:
data.head()

### Rozbicie daty na składowe

In [ ]:
def change_date(dataframe):
    dataframe['date'] = pd.to_datetime(dataframe['date'])
    dataframe['day'] = dataframe['date'].dt.dayofweek.astype(np.int64)
    dataframe['month'] = dataframe['month'].astype(np.int64)
    return dataframe


data, test = change_date(data), change_date(test)

### Wykres przedstawiający jakość powietrza w krajach afrykańskich

In [ ]:
sns.lineplot(data=data, x='date', y='pm2_5', hue='country')
plt.title('Jakość powietrza z podziałem na kraje')

### Wykres przedstawiający wartość pm2_5 w zarejestrowanych godzinach

In [ ]:
sns.barplot(data=data, x='hour', y='pm2_5', hue='country')
plt.title('Jakość powietrza w poszczególnych godzinach z podziałem na kraje')

### Wykres przedstawiający wartość pm2_5 z zależności od dnia tygodnia

In [ ]:
sns.barplot(data=data, x='day', y='pm2_5', hue='country')
plt.title('Jakość powietrza w każdym dniu tygodnia z podziałem na kraje')

### Wykres przedstawiający wartość pm2_5 z zależności od miesiąca

In [ ]:
sns.barplot(data=data, x='month', y='pm2_5', hue='country')
plt.title('Jakość powietrza w każdym dniu tygodnia z podziałem na kraje')

### Korelacje poszczególnych grup kolumn

In [ ]:
def correlation():
    for index, column in enumerate(starts_with):
        fig, ax = plt.subplots(figsize=(10, 10))
        selected_columns = [col for col in data.columns if col.startswith(column) or col == 'pm2_5']
        sns.heatmap(data[selected_columns].corr(), annot=True, fmt='.2f', cmap='viridis', ax=ax)
        plt.tight_layout()
        plt.show()


final_ids = test['id']
data.drop(columns=['id', 'city', 'country', 'site_id', 'date', 'site_latitude', 'site_longitude', 'sulphurdioxide_so2_column_number_density_15km'], inplace=True)
test.drop(columns=['id', 'city', 'country', 'site_id', 'date', 'site_latitude', 'site_longitude', 'sulphurdioxide_so2_column_number_density_15km'], inplace=True)
starts_with = data.columns.str.split('_', expand=True).levels[0].to_frame()
starts_with.drop(['month', 'day', 'hour', 'pm2'], inplace=True)
starts_with = starts_with[0].tolist()

correlation()

## <center>Czyszczenie danych</center>

### Wykresy przedstawiające ilość NaN w danej kolumnie w zależności od przyjętej jednostki czasu

In [ ]:
def plot_nans_based_on(date_unit='day'):
    for index, column_group in enumerate(starts_with):
        similar_columns = [el for el in data.columns if el.startswith(column_group)]
        fig, ax = plt.subplots(nrows=int(np.ceil(len(similar_columns) / 4)), ncols=4, figsize=(20, 20))
        for j, column in enumerate(similar_columns):
            x_cord, y_cord = divmod(j, 4)
            nans = data[[column, date_unit]].groupby(date_unit).apply(lambda x: x.isna().sum())
            nans.plot(kind='bar', x=date_unit, y=column, ax=ax[x_cord][y_cord])
        plt.show()


plot_nans_based_on()

### 1. Uzupełnienie wartości brakujących

In [ ]:
from sklearn.impute import KNNImputer

def fill_based_on(dataframe, date_unit='day'):
    date_range = dataframe[date_unit].unique()
    for date in date_range:
        for i, column in enumerate(starts_with):
            similar_columns = [el for el in dataframe.columns if el.startswith(column)]
            df = dataframe.loc[dataframe[date_unit] == date, similar_columns]
            if not df.empty:
                dataframe.loc[dataframe[date_unit] == date, similar_columns] = imputers[i].fit_transform(df)
    return dataframe

def prepare_dataframe(dataframe):  # usuwamy kolumny o dużej liczbie wartości NaN
    to_drop = []
    for index, el in enumerate(dataframe.columns):
        if dataframe[el].isna().sum() / len(dataframe) >= 0.9:
            to_drop.append(el)
    dataframe.drop(to_drop, axis=1, inplace=True)
    return dataframe


imputers = [KNNImputer(n_neighbors=15, weights='distance') for _ in range(len(starts_with))]
data, test = prepare_dataframe(data), prepare_dataframe(test)
data, test = fill_based_on(data), fill_based_on(test)

### Wykresy pudełkowe wskazujące wartości odstające

In [ ]:
def plot_boxplots():
    for index, column_group in enumerate(starts_with):
        similar_columns = [col for col in data.columns if col.startswith(column_group)]
        rows = int(np.ceil(len(similar_columns) / 4))
        if rows >= 1:
            fig, ax = plt.subplots(nrows=rows, ncols=4, figsize=(20, 20))
            fig.suptitle(column_group, fontsize=15)
            for j, column in enumerate(similar_columns):
                x_cord, y_cord = divmod(j, 4)
                data[column].plot(kind='box', ax=ax[x_cord][y_cord])
            plt.show()


plot_boxplots()

### 2. Usunięcie wartości odstających

In [ ]:
from scipy.stats import zscore

def del_big_outliers(dataframe, column):
    vec, indexes = zscore(dataframe[column]), []
    for index in range(len(vec)):
        if -3 <= vec[index] >= 3:
            indexes.append(index)
    dataframe.drop(index=indexes, inplace=True)
    return dataframe


# data = del_big_outliers(data, 'sulphurdioxide_so2_column_number_density')

data.info()

In [ ]:
data.head()

## <center>Selekcja cech</center>

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression

def select_best_features(not_fitted_X, not_fitted_y, num_of_features):
    sc = SelectKBest(score_func=mutual_info_regression, k=num_of_features)
    sc.fit(not_fitted_X, not_fitted_y)
    return sc

def plot_selection_scores(sc, num_of_features):
    scores = dict(zip(sc.feature_names_in_, sc.scores_))
    scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:num_of_features]
    scores_df = pd.DataFrame(scores, columns=['Feature', 'Score'])
    
    scores_df.plot(kind='bar', x='Feature', y='Score', figsize=(10, 6), rot=90, title='Oceny wybranych cech')
    plt.xlabel('Cecha')
    plt.ylabel('Ocena')


X, y = data.drop(['pm2_5'], axis=1), data['pm2_5']
# k = 30
# selector = select_best_features(X, y, k)
# X, test = selector.transform(X), selector.transform(test)
# 
# plot_selection_scores()

## <center>Transformacja danych</center>

### Potrzebne importy

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split

### 1. Wybór sposobu preprocessingu danych

In [ ]:
scaler = StandardScaler()

X = scaler.fit_transform(X, y)
test = scaler.transform(test)

### 2. Podział na zbiór testowy i treningowy

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=4)

## <center>Część obliczeniowa</center>

### Potrzebne importy

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import root_mean_squared_error

### Otrzymanie najlepszych parametrów

In [ ]:
def give_the_best(clf):
    gs = GridSearchCV(clf, params, scoring='neg_root_mean_squared_error', n_jobs=-1, cv=3, verbose=1)
    gs.fit(X_train, y_train)
    return gs.best_estimator_

def save_to_csv(y_pred, save_as):
    final_df = pd.concat([final_ids, pd.DataFrame.from_dict({'pm2_5': y_pred})], axis=1)
    final_df.to_csv(f'result\\{save_as}', index=False)

### <center>Regresja przy użyciu lasu</center>

In [ ]:
params = {
    'n_estimators': [700],
    'max_depth': np.arange(4, 11),
    'bootstrap': [True],
    'n_jobs': [-1],
    'random_state': [4],
    'warm_start': [True],
    'oob_score': [True],
    'ccp_alpha': np.linspace(0.001, 0.05, 7),
    # 'max_samples': np.arange(0.1, 1, 10)
}

rf = give_the_best(RandomForestRegressor())
save_to_csv(rf.predict(test), 'rf.csv')
print('Parametry lasu: ', rf.get_params())
print('RMSE: ', root_mean_squared_error(y_test, rf.predict(X_test)))

## <center>PyTorch</center>

### Potrzebne importy

In [ ]:
import torch
from torch import nn, optim

### 1. Wybór karty graficznej do nauki modelu

In [ ]:
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)

X_train_tensor = torch.tensor(X_train, device=device, dtype=torch.float)
X_test_tensor = torch.tensor(X_test, device=device, dtype=torch.float)
y_train_tensor = torch.tensor(y_train.to_numpy(), device=device, dtype=torch.float)
y_test_tensor = torch.tensor(y_test.to_numpy(), device=device, dtype=torch.float)
test_tensor = torch.tensor(test, device=device, dtype=torch.float)

### 2. Architektura sieci neuronowej

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(X_train_tensor.shape[1], 104),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(104, 270),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(270, 224),
            nn.SELU(),
            nn.Linear(224, 75),
            nn.Dropout(p=0.1),
            nn.SELU(),
            nn.Dropout(p=0.1),
            nn.Linear(75, 1),
        )
        
    def forward(self, x):
        return self.layers(x)

model = Net().to(device=device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(
    model.parameters(), 
    lr=0.01, 
    betas=(0.9, 0.999), 
    eps=1e-08, 
    weight_decay=1e-5,
    amsgrad=False, 
    fused=True
)

### 3. Nauka sieci neuronowej na zbiorze treningowym

In [ ]:
from torch.utils.data import DataLoader

model.train()

batch_size = 150
num_epochs = 100

final_train_tensor = torch.concat((X_train_tensor, y_train_tensor.unsqueeze(dim=1)), dim=1)
dataset = DataLoader(final_train_tensor, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for batch_idx, batch in enumerate(dataset):
        inputs, targets = batch[:, :-1], batch[:, -1]
        pred = model(inputs)
        optimizer.zero_grad()
        loss = loss_fn(pred.squeeze(), targets)
        loss.backward()
        optimizer.step()

### 4. Testowanie sieci neuronowej

In [ ]:
model.eval()

with torch.no_grad():
    pred = model(X_test_tensor)
    loss = loss_fn(pred.squeeze(), y_test_tensor)
    print(f'RMSE: {np.sqrt(loss.item()):.4f}')

## <center>Do wysłania</center>

In [ ]:
with torch.no_grad():
    final_pred = model(test_tensor)
    save_to_csv(final_pred.squeeze().numpy(force=True), 'nn.csv')

### Dodatkowe informacje
<ol>
    <li>The 15km SO2 band is ingested only when solar_zenith_angle < 70.</li>
    <li>Because of noise on the data, negative vertical column values are often observed in particular over clean regions or for low SO2 emissions. It is recommended not to filter these values except for outliers, i.e. for vertical columns lower than -0.001 mol/m^2.</li>
</ol>

In [ ]:
data[[col for col in data.columns if 'sensor_azimuth_angle' in col]].describe()

In [ ]:
data.info()